## Gun violence in America—looking at data
##### Measuring frequency distributions of some variables of interest in a few data sets

Note: The cells in this iPython notebook are scrollable if their contents extend beyond the frame. 

In [365]:
# Forces refresh of any cached variables that have been re-defined.
# Must come first.
from IPython import get_ipython
get_ipython().magic('reset -sf')

In [366]:
# Imports
import pandas as pd
import numpy as np
import os # for filepath
import re # for regular expressions

### US Mass Shootings, 1982-2018: Data From Mother Jones’ Investigation
https://www.motherjones.com/politics/2012/12/mass-shootings-mother-jones-full-data/f

In [367]:
# Loading the data
datapath = os.path.join('datasets', "")
mojo_df = pd.read_csv(datapath + 'mojo_us_mass_shoot_82_18.csv')

In [368]:
# A look at the columns and their data types
mojo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 22 columns):
Case                                   97 non-null object
Location                               97 non-null object
Date                                   97 non-null object
Year                                   97 non-null int64
Summary                                97 non-null object
Fatalities                             97 non-null int64
Injured                                97 non-null int64
Total victims                          97 non-null object
Venue                                  97 non-null object
Prior signs of mental health issues    97 non-null object
Mental health - details                86 non-null object
Weapons obtained legally               97 non-null object
Where obtained                         85 non-null object
Type of weapons                        97 non-null object
Weapon details                         91 non-null object
Race                          

In [369]:
# A look at the first few rows of data
mojo_df.head()

,Case,Location,Date,Year,Summary,Fatalities,Injured,Total victims,Venue,Prior signs of mental health issues,...,Where obtained,Type of weapons,Weapon details,Race,Gender,Sources,Mental Health Sources,latitude,longitude,Type
0,Stoneman Douglas High School shooting,"Parkland, Florida",2/14/18,2018,"Nikolas J. Cruz, 19, heavily armed with an AR-...",17,14,31,School,Yes,...,A Florida pawn shop,semiautomatic rifle,AR-15,White,M,https://www.nytimes.com/2018/02/14/us/parkland...,https://www.nytimes.com/2018/02/15/us/nikolas-...,NaN,NaN,Mass
1,Pennsylvania carwash shooting,"Melcroft, PA",1/28/18,2018,"Timothy O'Brien Smith, 28, wearing body armor ...",4,1,5,Other,TBD,...,TBD,semiautomatic rifle and semiautomatic handgun,NaN,White,M,http://www.wpxi.com/news/top-stories/family-me...,NaN,NaN,NaN,Mass
2,Rancho Tehama shooting spree,"Rancho Tehama, CA",11/14/17,2017,"Kevin Janson Neal, 44, went on an approximatel...",5,10,15,Other,TBD,...,TBD,semiautomatic rifles,Two illegally modified rifles,White,M,https://www.nbcnews.com/news/us-news/californi...,NaN,NaN,NaN,Spree
3,Texas First Baptist Church massacre,"Sutherland Springs, TX",11/5/17,2017,"Devin Patrick Kelley, a 26-year-old ex-US Air ...",26,20,46+,Religious,Yes,...,Purchased in April 2016 from an Academy Sports...,semiautomatic rifle,Ruger AR-556; Kelley also possessed semiautoma...,White,M,https://www.washingtonpost.com/news/morning-mi...,http://www.expressnews.com/news/local/article/...,32.780105,-96.800008,Mass
4,Walmart shooting in suburban Denver,"Thornton, CO",11/1/17,2017,"Scott Allen Ostrem, 47, walked into a Walmart ...",3,0,3,Other,Unclear,...,NaN,semiautomatic handgun,NaN,White,M,https://www.nytimes.com/2017/11/01/us/thornton...,NaN,43.060567,-88.106479,Mass


##### Frequency of mass shooting per state (1982 - 2018)
In order to do a frequency count on state data we need to isolate the state from the 'Location' column with a consistent representation (i.e. full name or abreviation). Creating two new columns: 'State_code' with two letter abreviations, and 'State' with full name.


In [370]:
# Create two maps: (1) state to abreviation, (2) abreviation to state
state_abrv = {'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}
abrv_state = {v: k for k, v in state_abrv.items()} # reverse mapping

In [371]:
# Transform 'Location' data into two new columns: 'State' and 'State_code'
mojo_df['State_code'] = mojo_df['Location'].str.split(',').str.get(1).str.strip()
mojo_df['State'] = mojo_df['Location'].str.split(',').str.get(1).str.strip()
mojo_df = mojo_df.replace({'State_code': state_abrv})
mojo_df = mojo_df.replace({'State': abrv_state})

Scroll right to examine new columns for state names and state abreviations: State, State_code.

In [372]:
mojo_df.head()

,Case,Location,Date,Year,Summary,Fatalities,Injured,Total victims,Venue,Prior signs of mental health issues,...,Weapon details,Race,Gender,Sources,Mental Health Sources,latitude,longitude,Type,State_code,State
0,Stoneman Douglas High School shooting,"Parkland, Florida",2/14/18,2018,"Nikolas J. Cruz, 19, heavily armed with an AR-...",17,14,31,School,Yes,...,AR-15,White,M,https://www.nytimes.com/2018/02/14/us/parkland...,https://www.nytimes.com/2018/02/15/us/nikolas-...,NaN,NaN,Mass,FL,Florida
1,Pennsylvania carwash shooting,"Melcroft, PA",1/28/18,2018,"Timothy O'Brien Smith, 28, wearing body armor ...",4,1,5,Other,TBD,...,NaN,White,M,http://www.wpxi.com/news/top-stories/family-me...,NaN,NaN,NaN,Mass,PA,Pennsylvania
2,Rancho Tehama shooting spree,"Rancho Tehama, CA",11/14/17,2017,"Kevin Janson Neal, 44, went on an approximatel...",5,10,15,Other,TBD,...,Two illegally modified rifles,White,M,https://www.nbcnews.com/news/us-news/californi...,NaN,NaN,NaN,Spree,CA,California
3,Texas First Baptist Church massacre,"Sutherland Springs, TX",11/5/17,2017,"Devin Patrick Kelley, a 26-year-old ex-US Air ...",26,20,46+,Religious,Yes,...,Ruger AR-556; Kelley also possessed semiautoma...,White,M,https://www.washingtonpost.com/news/morning-mi...,http://www.expressnews.com/news/local/article/...,32.780105,-96.800008,Mass,TX,Texas
4,Walmart shooting in suburban Denver,"Thornton, CO",11/1/17,2017,"Scott Allen Ostrem, 47, walked into a Walmart ...",3,0,3,Other,Unclear,...,NaN,White,M,https://www.nytimes.com/2017/11/01/us/thornton...,NaN,43.060567,-88.106479,Mass,CO,Colorado


**Let's now look at frequency counts and distributions (percentages) of mass shootings per state from 1982-2018.**

In [373]:
# Default sort by value column in decending order.
mass_shoot_counts = mojo_df['State'].value_counts()
print(type(mass_shoot_counts))
mass_shoot_counts

<class 'pandas.core.series.Series'>


California        16
Florida           10
Texas              8
Washington         7
Colorado           6
New York           4
Wisconsin          4
Pennsylvania       3
Connecticut        3
South Carolina     2
Nevada             2
Oregon             2
Michigan           2
North Carolina     2
Minnesota          2
Illinois           2
Kentucky           2
Ohio               2
Georgia            2
Maryland           1
Louisiana          1
Virginia           1
Mississippi        1
Kansas             1
Iowa               1
Oklahoma           1
Arizona            1
Utah               1
Massachusetts      1
Arkansas           1
Hawaii             1
D.C.               1
Nebraska           1
Missouri           1
Tennessee          1
Name: State, dtype: int64

In [374]:
# Convert the Series to DataFrame. Name columns correctly.
mass_shoot_counts_df = mass_shoot_counts.to_frame()
print(type(mass_shoot_counts_df))
mass_shoot_counts_df = mass_shoot_counts_df.rename(columns = {'State':'MASS CNT'})
mass_shoot_counts_df['STNAME'] = mass_shoot_counts_df.index
mass_shoot_counts_df.set_index('STNAME', inplace=True)
mass_shoot_counts_df

<class 'pandas.core.frame.DataFrame'>


,MASS CNT
STNAME,
California,16
Florida,10
Texas,8
Washington,7
Colorado,6
New York,4
Wisconsin,4
Pennsylvania,3
Connecticut,3


In [375]:
mojo_df['State'].value_counts(normalize=True)

California        0.164948
Florida           0.103093
Texas             0.082474
Washington        0.072165
Colorado          0.061856
New York          0.041237
Wisconsin         0.041237
Pennsylvania      0.030928
Connecticut       0.030928
South Carolina    0.020619
Nevada            0.020619
Oregon            0.020619
Michigan          0.020619
North Carolina    0.020619
Minnesota         0.020619
Illinois          0.020619
Kentucky          0.020619
Ohio              0.020619
Georgia           0.020619
Maryland          0.010309
Louisiana         0.010309
Virginia          0.010309
Mississippi       0.010309
Kansas            0.010309
Iowa              0.010309
Oklahoma          0.010309
Arizona           0.010309
Utah              0.010309
Massachusetts     0.010309
Arkansas          0.010309
Hawaii            0.010309
D.C.              0.010309
Nebraska          0.010309
Missouri          0.010309
Tennessee         0.010309
Name: State, dtype: float64

**Note**: This task so far is in support of my original question about what association there might be between the number of mass shootings per state and the amount of NRA* contributions to congressional candidates per state. I have anticipated the need to correlate additional variables from multiple datasets in order to gain a broader picture of factors contributing to the controversy around gun violence, and right away I'm noticing that state population needs to be factored into any statistic used when making comparisons among stats. And so I think I will want to locate and include data on state populations.

It may also be that state-level consideration any association between mass shootings and NRA political contributions are compromised by population and or even physical size disparities. Perhaps analyzing such associations at the congressional district would be more appropriate in terms of comparing more-or-less equally sized units of measure. However, since Senators are elected at the state level, it might therefore be good to conduct analysis of comparison between mass-shootings and NRA contributions at both the state and congressional district level. I do, however, anticipate running into sparcity issues when examining mass-shootings at the congressional district level. A related question would be  what effect aggregating Senatorial and Congressional Representative contributions at the state level and looking at association measures between campaign contributions and mass shootings only at the state level. 

*National Rifle Association

### Annual Estimates of the Resident Population: 2016 Population Estimates (U.S. Census Bureau)
https://www.census.gov/data/datasets/2016/demo/popest/counties-total.html#ds

In [376]:
# Loading the data
est_pop_2016_df = pd.read_csv(
    datapath + 'co-est2016-alldata.csv', encoding='latin-1')

In [377]:
est_pop_2016_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3193 entries, 0 to 3192
Columns: 116 entries, SUMLEV to RNETMIG2016
dtypes: float64(36), int64(78), object(2)
memory usage: 2.8+ MB


There is a lot of data here (116 columns, 3193 rows), such as county populations and demographic breakdowns. I'm really just interested in state populations. I could have scraped this data in simpler form on the Internet, but I wanted to get it direction from the Census Bureau. Luckily, I was able to view a scrolling tabular view of the data online and I saw what I'm after, which are the populations of each state in the U.S.

In [378]:
# Columns of interest: state name, state population (estimate for 2016)
# The way this dataset is constructed, the state population is flaged by 'COUNTY' == 0.
state_pop_df = est_pop_2016_df.loc[est_pop_2016_df['COUNTY'] == 0, 
                                ['STNAME', 'POPESTIMATE2016']]
# Default sort by alphabetical index
state_pop_df.set_index('STNAME', inplace=True)
print(len(state_pop_df))
state_pop_df

51


,POPESTIMATE2016
STNAME,
Alabama,4863300
Alaska,741894
Arizona,6931071
Arkansas,2988248
California,39250017
Colorado,5540545
Connecticut,3576452
Delaware,952065
District of Columbia,681170


Now I would like to merge mojo_df and state_pop and then experiment with *normalizing*—an unfortunate choice of terms, perhaps—the number of mass shooting in each state by the population of the state. I am not sure what implication this may have, but I consider it to be part of the EDA\*. I invite thoughts and comments by any readers. For instance, do I want to distribute shootings over the population? Or distribute the population over the shootings. Intuitively, I should think it is the former, but I will calculate both below.

\*Exploratory Data Analysis

In [379]:
state_pop_mass_df = pd.merge(mass_shoot_counts_df, state_pop_df, 
                             left_index=True, right_index=True)
print(len(state_pop_mass_df))
# Default decending sort by number of mass shootings
state_pop_mass_df

34


,MASS CNT,POPESTIMATE2016
STNAME,,
California,16,39250017
Florida,10,20612439
Texas,8,27862596
Washington,7,7288000
Colorado,6,5540545
New York,4,19745289
Wisconsin,4,5778708
Pennsylvania,3,12784227
Connecticut,3,3576452


**Let's order (sort) the table by population**

In [380]:
# Sort by population
state_pop_mass_df.sort_values('POPESTIMATE2016', ascending=False)

,MASS CNT,POPESTIMATE2016
STNAME,,
California,16,39250017
Texas,8,27862596
Florida,10,20612439
New York,4,19745289
Illinois,2,12801539
Pennsylvania,3,12784227
Ohio,2,11614373
Georgia,2,10310371
North Carolina,2,10146788


**Decending sort of population per shooting fatality**

In [381]:
# Adding column for ratio of population to mass shootings
state_pop_mass_df['POP OVER MASS CNT'] = state_pop_mass_df['POPESTIMATE2016'] / state_pop_mass_df['MASS CNT']
# Descending sort of population over number of mass shootings. I.e. decending population per shooting fatality
state_pop_mass_df.sort_values('POP OVER MASS CNT', ascending=False)

,MASS CNT,POPESTIMATE2016,POP OVER MASS CNT
STNAME,,,
Virginia,1,8411808,8.411808e+06
Arizona,1,6931071,6.931071e+06
Massachusetts,1,6811779,6.811779e+06
Tennessee,1,6651194,6.651194e+06
Illinois,2,12801539,6.400770e+06
Missouri,1,6093000,6.093000e+06
Maryland,1,6016447,6.016447e+06
Ohio,2,11614373,5.807186e+06
Georgia,2,10310371,5.155186e+06


**Let's sort the table once again, by the ratio of number of shootings to population.**

These sorts above and the sort below are inverses of each other. This also means that the following table could be achieved by simply reversing the sort order of the one above.

In [382]:
# Adding column for ratio of number of mass shootings to population
state_pop_mass_df['MASS CNT OVER POP'] = state_pop_mass_df['MASS CNT'] / state_pop_mass_df['POPESTIMATE2016']
# Descending sort of  mass shootings over population. I.e. decending shooting fatalities per population.
state_pop_mass_df.sort_values('MASS CNT OVER POP', ascending=False)

,MASS CNT,POPESTIMATE2016,POP OVER MASS CNT,MASS CNT OVER POP
STNAME,,,,
Colorado,6,5540545,9.234242e+05,1.082926e-06
Washington,7,7288000,1.041143e+06,9.604830e-07
Connecticut,3,3576452,1.192151e+06,8.388201e-07
Hawaii,1,1428557,1.428557e+06,7.000071e-07
Wisconsin,4,5778708,1.444677e+06,6.921962e-07
Nevada,2,2940058,1.470029e+06,6.802587e-07
Nebraska,1,1907116,1.907116e+06,5.243520e-07
Oregon,2,4093465,2.046732e+06,4.885836e-07
Florida,10,20612439,2.061244e+06,4.851439e-07
